<a href="https://colab.research.google.com/github/murilofarias10/Python/blob/main/SmartFillQuiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Fill in the Blank Quiz Generator - Murilo Farias**


The application allows users to upload plain text (.txt) files and generates quiz questions from the content, This tool is designed to combine technology with education.
By dynamically generating questions based on text analysis, it not only promotes language learning but also serves as an innovative resource for teachers, students, or anyone looking to enhance their vocabulary and comprehension skills.


**Importing Libraries:**

Gradio (gr): A Python library to build user-friendly web interfaces for machine learning or Python scripts. Here, it’s used to create an interactive quiz generator.

NLTK (nltk): The Natural Language Toolkit is a suite of libraries for text processing, like tokenization, stopword removal, and part-of-speech tagging.
Random (random): To add randomness, such as selecting a random sentence or shuffling answer options.

OS (os): To manage file paths and directories for saving or accessing files.

**What’s happening here?**

**1.   NLTK Resources:**
*   required data files (punkt, averaged_perceptron_tagger, and stopwords) are
downloaded into a specific directory.
*   punkt: Used for tokenizing sentences into words.
*   averaged_perceptron_tagger: Tags words with parts of speech (e.g., noun, verb).
*   stopwords: Filters out common words.


**2.   Validating the TXT File:**
*   validate the uploaded file to ensure it’s a .txt file and isn’t empty.

**3.   Generating Quiz Questions:**
*   Sentence Selection: Sentences are filtered based on length
*   Word Extraction: Words from the sentence are filtered
*   Blank Creation: A random word is replaced with a blank to form the quiz question.
*   Option Generation: The correct word and two random re selected as answer choices.
*   Shuffling: All answer choices are shuffled to randomize their order

**4. Process File and Generate Output**
*   The selected sentence is formatted into a "fill-in-the-blank" question with multiple-choice options.
*   The correct answer is embedded in a collapsible HTML tag for users to reveal after attempting the question.

**5. Creating the Gradio Interface**
* File Upload: Allows users to upload their text files
* Buttons: "New Question" generates a question, while "Other File" clears the interface for a new file.
* Custom CSS: Improves the appearance of the interface with styling for text, buttons, and question boxes.
* Output Section: Displays the generated question along with the options

**6. Launching the Application**
* The demo.launch command starts the Gradio app, hosting it locally or on the web. Users can interact with the app in their browser.







In [ ]:
pip install gradio PyPDF2 python-pptx

In [ ]:
pip install gradio nltk

In [9]:
import gradio as gr
import nltk
from nltk.tokenize import word_tokenize #>>>> word_tokenize: Tokenizes text into words.
from nltk.corpus import stopwords #>>>> stopwords: Removes common words that don’t add value to the quiz.
from nltk.tag import pos_tag #>>> pos_tag: (Optional) Could be used to identify nouns, verbs, etc., for smarter word selection.
import random
import os
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# --- 1 NLTK data ---
# Create a directory for NLTK data if it doesn't exist and download necessary resources
nltk_data_dir = os.path.expanduser('~/nltk_data')
if not os.path.exists(nltk_data_dir): #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> path.exists: Checks if a path exists
    os.makedirs(nltk_data_dir) #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> makedirs: Creates directories if they don’t exist.
    for resource in ['punkt', 'averaged_perceptron_tagger', 'stopwords']:
        nltk.download(resource, quiet=True)


#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# --- 2 Validate TXT File ---
def read_txt_file(file):
    """
    Reads the uploaded TXT file and ensures it's valid.
    Returns the content or an error message.
    """
    try:
        if not file or not file.name.lower().endswith('.txt'):
            return "Error: Please upload a valid TXT file."
        with open(file.name, 'r', encoding='utf-8') as f:
            content = f.read().strip()
        return content if content else "Error: Empty file."
    except Exception as e:
        return f"Error reading file: {str(e)}"

#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# --- 3 Generate Create Quiz Questions ---
def create_quiz_questions(content):
    """
    Processes the content and generates a single 'fill-in-the-blank' question.
    Includes multiple-choice options and identifies the correct answer.
    """
    if not content or isinstance(content, str) and content.startswith("Error"):
        return "Could not generate question. Please check your file."

    # Filter sentences suitable for creating questions
    sentences = [s.strip() for s in content.split('\n') if len(s.strip()) > 20 and ':' not in s]

    if not sentences:
        return "Not enough content to generate a question."

    # Gather words for creating blank and distractor options
    all_words = set(word.lower() for sentence in sentences for word in sentence.split()
                    if len(word) > 3 and not word.isupper())

    # Randomly select a sentence >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> choice: Selects a random item (e.g., a sentence).
    sentence = random.choice(sentences)

    # Filter words suitable for blanks
    words = [word for word in sentence.split()
             if len(word) > 3 and not word.isupper()
             and word.lower() not in ['they', 'their', 'them', 'like', 'with', 'from']]

    if not words:
        return "Could not generate a valid question from the content."

    # Create a blank in the sentence
    chosen_word = random.choice(words)
    blank_sentence = sentence.replace(chosen_word, "_____", 1)

    # Select similar words for distractor options
    similar_words = [w for w in all_words
                     if len(w) >= len(chosen_word) - 2 and len(w) <= len(chosen_word) + 2 and w != chosen_word.lower()]

    if len(similar_words) < 2:
        return "Could not generate enough options."

    # Shuffle options (correct word + distractors) >>>>>>>>>>>>>>>>>>>>>>>>ample: Picks random items (e.g., distractor words).
    wrong_options = random.sample(similar_words, 2)
    options = [chosen_word] + wrong_options
    random.shuffle(options) #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> shuffle: Randomizes the order of answer options.

    return {
        "question": blank_sentence,
        "options": [f"Option {chr(97 + i)}: {opt}" for i, opt in enumerate(options)],
        "answer": chosen_word,
        "correct_option": f"Option {chr(97 + options.index(chosen_word))}"
    }


#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# --- 4 Process File and Generate Output ---
def process_file(file):
    """
    Reads the file, generates a question, and formats the output.
    """
    content = read_txt_file(file)
    if isinstance(content, str) and content.startswith("Error"):
        return content

    question = create_quiz_questions(content)
    if isinstance(question, str):
        return question

    # Format output for display in the interface
    output = f"### Fill in the blank:\n{question['question']}\n\n"
    output += "\n".join(question['options']) + "\n"
    output += f"\n<details><summary>👉 Show Answer</summary>\n\n"
    output += f"**{question['answer']} ({question['correct_option']})**\n"
    output += "</details>\n"

    return output

#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# --- 5 Creating the Gradio Interface ---
def create_interface():
    """
    Builds the Gradio interface with custom styling for the quiz. >>>>>>>>>>>> Blocks: Creates a flexible layout for the app
    """
    with gr.Blocks(css="""
        /* Custom CSS for quiz styling */
        .question-box {
            font-size: 14px !important;
            line-height: 1.2 !important;
            white-space: pre-wrap !important;
            padding: 10px !important;
            background-color: #2e7d32 !important;
            border-radius: 5px !important;
            color: white !important;
        }
        details {
            padding: 3px;
            background-color: #2e7d32;
            border-radius: 3px;
            margin: 5px 0;
            border: 1px solid #1b5e20;
        }
        summary {
            cursor: pointer;
            color: white;
            font-weight: bold;
            padding: 3px;
            background-color: #2e7d32;
            border-radius: 2px;
        }
        summary:hover { background-color: #1b5e20; }
        details[open] { padding-bottom: 4px; }
        details > *:not(summary) {
            color: white !important;
            background-color: #1b5e20;
            padding: 4px;
            border-radius: 2px;
            margin-top: 3px;
        }
        .main-title {
            font-size: 20px !important;
            font-weight: bold !important;
            color: #2e7d32 !important;
            margin: 0 !important;
            padding: 5px 0 !important;
        }
    """) as demo:
        gr.Markdown("# 📚 Fill in the Blank Quiz", elem_classes=["main-title"])

        # Input file section and action buttons >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> File: Allows users to upload .txt files.
        with gr.Row():
            file_input = gr.File(label="Upload TXT file", file_types=[".txt"], scale=2)
            with gr.Column(scale=1):
                with gr.Row(elem_classes=["button-row"]):
                    submit_btn = gr.Button("New Question", variant="primary", size="sm")
                    clear_btn = gr.Button("Other File", size="sm")

        # Output display section >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> Markdown: Displays formatted text (questions, options, answers).
        output = gr.Markdown(elem_classes=["question-box"])

        # Link buttons to functions >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> Button: Handles user actions (e.g., generating questions).
        submit_btn.click(fn=process_file, inputs=[file_input], outputs=[output])
        clear_btn.click(fn=lambda: (None, None), inputs=[], outputs=[file_input, output])

    return demo

#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

# --- 6 Launching the Application ---
if __name__ == "__main__":
    # Launch the Gradio app >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> launch(): Starts the app
    demo = create_interface()
    demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://84a671eceda113b039.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
